In [ ]:
!pip install python-docx scikit-learn

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving E20047 CHANDIMAL P.A.K.D. MR_66663_assignsubmission_file_Mahaweli_Dams_and_Minipe_Report_E20_047.pdf to E20047 CHANDIMAL P.A.K.D. MR_66663_assignsubmission_file_Mahaweli_Dams_and_Minipe_Report_E20_047.pdf
Saving E20398 THILAKSHANA P.K.A. MR_66630_assignsubmission_file_Irrigation_field_visit_report_E20398.pdf to E20398 THILAKSHANA P.K.A. MR_66630_assignsubmission_file_Irrigation_field_visit_report_E20398.pdf
Saving E20002 ABEYKOON C.S. MR_66681_assignsubmission_file_Irrigation field visit Previsit Assignment. E20002.pdf to E20002 ABEYKOON C.S. MR_66681_assignsubmission_file_Irrigation field visit Previsit Assignment. E20002.pdf
Saving E20225 MADHURANDA A.K.S. MR_66676_assignsubmission_file_Irrigation field visit Previsit Assignment. E20225.pdf to E20225 MADHURANDA A.K.S. MR_66676_assignsubmission_file_Irrigation field visit Previsit Assignment. E20225.pdf
Saving E20206 KULASEKARA P.M.N. MR_66636_assignsubmission_file_E20206_preAssignment.pdf to E20206 KULASEKARA P.M.N. MR_66636_a

In [ ]:
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 78.6 MB/s eta 0:00:00


In [ ]:
import fitz  # from PyMuPDF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import io

def extract_text_from_pdf(file_bytes):
    text = ""
    with fitz.open(stream=file_bytes, filetype="pdf") as doc:
        for page in doc:
            text += page.get_text()
    return text.strip()

file_texts = []
file_names = []

for filename, file_bytes in uploaded.items():
    text = extract_text_from_pdf(file_bytes)
    print(f"\n--- {filename} ---\n{text[:300]}")  # Preview
    if text:
        file_texts.append(text)
        file_names.append(filename)

if len(file_texts) < 2:
    print("\n❌ Not enough valid PDF documents with extractable text.")
else:
    # TF-IDF Vectorization + Similarity
    vectorizer = TfidfVectorizer(stop_words='english').fit_transform(file_texts)
    similarity_matrix = cosine_similarity(vectorizer)

    # Threshold to flag similar files
    threshold = 0.95
    print("\n📋 Highly similar PDF document pairs (Possible copies):")
    found = False
    for i in range(len(file_names)):
        for j in range(i + 1, len(file_names)):
            score = similarity_matrix[i][j]
            if score > threshold:
                print(f"{file_names[i]} <--> {file_names[j]} : {score:.2f}")
                found = True
    if not found:
        print("No highly similar files found above threshold.")



--- E20047 CHANDIMAL P.A.K.D. MR_66663_assignsubmission_file_Mahaweli_Dams_and_Minipe_Report_E20_047.pdf ---
Report: Dams on Mahaweli River up to Minipe Anicut and Details of Minipe Scheme
1. Types of Dams and Their Specifications in the Mahaweli River up to Minipe Anicut
The Mahaweli River, the longest river in Sri Lanka, has several major dam structures developed under the
Mahaweli Development Program. U

--- E20398 THILAKSHANA P.K.A. MR_66630_assignsubmission_file_Irrigation_field_visit_report_E20398.pdf ---
Mahaweli River Headworks and Minipe Diversion System
1. Dams in the Mahaweli River up to Minipe Anicut
The Mahaweli River features a cascade of major hydraulic structures upstream of the Minipe Anicut, forming
a critical water and energy infrastructure system in Sri Lanka. The main dams in this casc

--- E20002 ABEYKOON C.S. MR_66681_assignsubmission_file_Irrigation field visit Previsit Assignment. E20002.pdf ---
Types of Dams and Specifications in the Mahaweli River up to Mini

In [11]:
# Create and save a text report of similar files
report_lines = []
report_lines.append("📋 Highly Similar PDF Document Pairs (Threshold = 0.95)\n")

found = False
for i in range(len(file_names)):
    for j in range(i + 1, len(file_names)):
        score = similarity_matrix[i][j]
        if score > threshold:
            found = True
            report_lines.append(f"{file_names[i]} <--> {file_names[j]} : {score:.2f}")

if not found:
    report_lines.append("No highly similar files found above the threshold.")

# Save the report to a .txt file
with open("similarity_report.txt", "w") as f:
    f.write("\n".join(report_lines))

# Offer download
from google.colab import files
files.download("similarity_report.txt")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>